In [1]:
import os, sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [6]:
from scripts.data_load import load_data
from scripts.preprocess import preprocess_data
from scripts.data_visualization import *
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import ipaddress
import joblib
from tensorflow import keras

In [7]:
# Load data
fraud_data = load_data('../data/cleaned_fraud_data.csv')
credit_data = load_data('../data/creditcard.csv')
ip_country = load_data('../data/IpAddress_to_Country.csv')

In [8]:
# Load models
cnn_model = keras.models.load_model("../models/cnn_model.h5")
random_model = joblib.load("../models/random_forest_model.pkl")
rnn_model = keras.models.load_model("../models/rnn_model.h5")
lstms_model = keras.models.load_model("../models/lstm_model.h5")

In [9]:
# Fraud_Data.csv
X_fraud = fraud_data.drop(columns=['device_id', 'class', 'country'])
y_fraud = fraud_data['class']

# creditcard.csv
X_credit = credit_data.drop(columns=['Class'])
y_credit = credit_data['Class']

In [10]:

# Fraud_Data.csv
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

# creditcard.csv
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(
    X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit
)

X_train_fraud['purchase_time'] = pd.to_datetime(X_train_fraud['purchase_time'])
X_train_fraud['purchase_time'] = X_train_fraud['purchase_time'].astype(int) / 10**9  # Convert to seconds

X_test_fraud['purchase_time'] = pd.to_datetime(X_test_fraud['purchase_time'])
X_test_fraud['purchase_time'] = X_test_fraud['purchase_time'].astype(int) / 10**9  # Convert to seconds


X_train_fraud['signup_time'] = pd.to_datetime(X_train_fraud['signup_time'])
X_train_fraud['signup_time'] = X_train_fraud['signup_time'].astype(int) / 10**9  # Convert to seconds

X_test_fraud['signup_time'] = pd.to_datetime(X_test_fraud['signup_time'])
X_test_fraud['signup_time'] = X_test_fraud['signup_time'].astype(int) / 10**9  # Convert to seconds


In [ ]:
import shap

# Ensure all columns are numeric
X_train_fraud = X_train_fraud.apply(pd.to_numeric, errors='coerce').fillna(0)
X_test_fraud = X_test_fraud.apply(pd.to_numeric, errors='coerce').fillna(0)

# Convert boolean columns to integers
X_train_fraud = X_train_fraud.astype({col: 'int' for col in X_train_fraud.select_dtypes(include='bool').columns})
X_test_fraud = X_test_fraud.astype({col: 'int' for col in X_test_fraud.select_dtypes(include='bool').columns})

# Initialize SHAP explainer
explainer = shap.Explainer(lstms_model, X_train_fraud)
shap_values = explainer(X_test_fraud)

# SHAP Summary Plot
shap.summary_plot(shap_values, X_test_fraud, plot_type="bar")

PermutationExplainer explainer:   3%|▎         | 801/30223 [2:09:14<19:43:28,  2.41s/it]   